# XAI Experiment Data Cleaning

In [3]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# from pandas.io.json import json_normalize

In [4]:
# load JSON
def load_data(filename):
    df = pd.read_json(filename)
    data_sorted = df['formAnswers'].apply(pd.Series).join(df[['swymerId', 'workDurationInSeconds']])
    return data_sorted.fillna("")

In [5]:
# from google.colab import files

# uploaded = files.upload()

In [48]:
#load json files
v1_q = load_data('data/answers-XAI2-v1-basic-group-2-qualification.json')
v1_m = load_data('data/answers-XAI2-v1-basic-group-2-main.json')

v2_q = load_data('data/answers-XAI2-v2-salient-group-2-qualification.json')
v2_m = load_data('data/answers-XAI2-v2-salient-group-2-main.json')

# v2_q_p2 = load_data('data/answers-XAI2-v2-salient-group-2-qualification-part2.json')
# v2_m_p2 = load_data('data/answers-v2-salient-group-2-main-part2.json')

# v2_q = pd.concat([v2_q_p1, v2_q_p2])
# v2_m = pd.concat([v2_m_p1, v2_m_p2])

v3_q = load_data('data/answers-XAI2-v3-explanations-group-2-qualification.json')
v3_m = load_data('data/answers-XAI2-v3-explanations-group-2-main.json')

In [41]:
df_v1_q[["newsitem.7.control-question", "newsitem.8.control-question"]]
# df_v1_q.info

,newsitem.7.control-question,newsitem.8.control-question
0,wrong-1,
1,wrong-1,
2,correct,correct
3,correct,wrong-1
4,wrong-2,
5,correct,correct
6,,wrong-2
7,correct,correct
8,correct,correct
9,correct,correct


In [49]:
#confert to pandas data frame
df_v1_q = pd.DataFrame(v1_q)
df_v1_m = pd.DataFrame(v1_m)

df_v2_q = pd.DataFrame(v2_q)
df_v2_m = pd.DataFrame(v2_m)

df_v3_q = pd.DataFrame(v3_q)
df_v3_m = pd.DataFrame(v3_m)


In [50]:
# remove columns from df_XX_q which are redundant or not needed for the analysis
reduntant_columns = [
  "METADATA.FEATURE",
  "METADATA.GROUP",
  "confirm.0",
]

df_v1_q = df_v1_q.drop(columns=reduntant_columns)
df_v2_q = df_v2_q.drop(columns=reduntant_columns)
df_v3_q = df_v3_q.drop(columns=reduntant_columns)

# get a list of all column names starting with "tutorial."
tutorial_columns = [col for col in df_v1_q.columns if col.startswith("tutorial.")]
tutorial_columns.append("understand-task")

# remove tutorial columns from both dataframes -> Only qualif
df_v1_q = df_v1_q.drop(columns=tutorial_columns)
# df_v1_m = df_v1_m.drop(columns=tutorial_columns)

df_v2_q = df_v2_q.drop(columns=tutorial_columns)
# df_v2_m = df_v2_m.drop(columns=tutorial_columns)

df_v3_q = df_v3_q.drop(columns=tutorial_columns)
# df_v3_m = df_v3_m.drop(columns=tutorial_columns)

# remove "METADATA.PART" columns from both dataframes
df_v1_q = df_v1_q.drop(columns=["METADATA.PART"])
df_v1_m = df_v1_m.drop(columns=["METADATA.PART"])

df_v2_q = df_v2_q.drop(columns=["METADATA.PART"])
df_v2_m = df_v2_m.drop(columns=["METADATA.PART"])

df_v3_q = df_v3_q.drop(columns=["METADATA.PART"])
df_v3_m = df_v3_m.drop(columns=["METADATA.PART"])

In [51]:
# merge df_v1_q and df_v1_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v1 = pd.merge(df_v1_q, df_v1_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v1

,personal-code,newsitem.7.rating-before-xai,newsitem.7.rating-after-xai,POINTS.qualification,newsitem.7.control-question,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,19713,-1,-1,0,wrong-1,,,,,,...,5,3,5,wrong-1,5,3,-1,-1,2,178
1,svm73,73,62,0,wrong-1,,,,,,...,4,4,5,wrong-1,4,4,69,65,4,432
2,VJT09,65,84,0,wrong-2,,,,,,...,6,6,6,correct,5,7,74,-1,6,1178
3,SKI877,56,49,2,correct,6,-1,6,6,6,...,4,3,4,correct,5,5,41,-1,4,1127
4,AHL41,61,82,2,correct,7,-1,7,7,7,...,6,6,6,correct,6,6,85,-1,6,702


In [54]:
df_v2

,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,personal-code,newsitem.7.rating-before-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,newsitem.8.system-evaluation.classified-correctly,newsitem.7.system-evaluation.xai-features-useful,newsitem.8.system-evaluation.xai-features-useful,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,6,-1,SJS97,67,6,6,7,7,6,4,...,3,4,5,correct,4,3,58,-1,4,796
1,7,-1,FDA73,66,7,7,7,3,7,2,...,7,7,7,correct,7,7,58,-1,7,3610
2,7,-1,jrc88,86,7,7,7,7,7,7,...,7,7,6,correct,7,7,86,15,7,691
3,7,-1,JDP35,54,7,7,7,7,7,7,...,6,6,4,wrong-2,2,6,51,-1,6,590
4,6,-1,DLL27,83,6,6,6,2,6,2,...,6,5,5,correct,4,6,75,-1,5,980


In [55]:
# dataframe of participants who did the qualification task for v1 and the main task for v2
pd.merge(df_v1_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))

,personal-code,newsitem.7.rating-before-xai,newsitem.7.rating-after-xai,POINTS.qualification,newsitem.7.control-question,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main


In [56]:
# dataframe of participants who are in the main task of v2, but not in either qualification task
df_v2_m[~df_v2_m.swymerId.isin(df_v1_q.swymerId) & ~df_v2_m.swymerId.isin(df_v2_q.swymerId)]

,information-literacy.identify-sdg,newsitem.4.system-evaluation.indications-useful-concreteness,newsitem.6.system-evaluation.indications-useful,news-dashboard-evaluation.assist-decisions,newsitem.1.control-question,ai-system-evaluation.most-useful-explanation-feature,newsitem.5.system-evaluation.xai-features-useful,newsitem.1.system-evaluation.xai-features-useful,newsitem.2.rating-before-xai,ai-system-evaluation.easy-to-follow-what-system-does,...,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,swymerId,workDurationInSeconds


In [57]:
# merge df_v2_q and df_v2_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v2 = pd.merge(df_v2_q, df_v2_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v2

,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.rating-after-xai,personal-code,newsitem.7.rating-before-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,newsitem.8.system-evaluation.classified-correctly,newsitem.7.system-evaluation.xai-features-useful,newsitem.8.system-evaluation.xai-features-useful,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,6,-1,SJS97,67,6,6,7,7,6,4,...,3,4,5,correct,4,3,58,-1,4,796
1,7,-1,FDA73,66,7,7,7,3,7,2,...,7,7,7,correct,7,7,58,-1,7,3610
2,7,-1,jrc88,86,7,7,7,7,7,7,...,7,7,6,correct,7,7,86,15,7,691
3,7,-1,JDP35,54,7,7,7,7,7,7,...,6,6,4,wrong-2,2,6,51,-1,6,590
4,6,-1,DLL27,83,6,6,6,2,6,2,...,6,5,5,correct,4,6,75,-1,5,980


In [58]:
df_v3 = pd.merge(df_v3_q, df_v3_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))
df_v3

,newsitem.8.rating-after-xai,personal-code,newsitem.8.control-question,newsitem.8.rating-before-xai,POINTS.qualification,newsitem.7.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.7.rating-before-xai,newsitem.7.system-evaluation.understand-what-system-does,newsitem.7.system-evaluation.indications-useful,newsitem.7.system-evaluation.classified-correctly,...,ai-system-evaluation.understand-why-system-provided-decision,ai-system-evaluation.system-criteria-acceptable,information-literacy.research-level-skilss,newsitem.4.control-question,information-literacy.have-it-skills,ai-system-evaluation.benefit-of-doubt,newsitem.1.rating-before-xai,newsitem.4.rating-after-xai,ai-system-evaluation.system-decides-consistently,workDurationInSeconds.main
0,-1,Pzf94,correct,-1,2,6,67,7,7,7,...,6,5,6,correct,4,5,62,-1,5,2277
1,55,X,correct,-1,2,5,87,5,6,6,...,6,4,4,wrong-2,3,4,75,53,7,327
2,-1,X,wrong-1,-1,0,,,,,,...,6,7,6,correct,7,6,-1,-1,6,1273
3,23,1955,correct,-1,2,7,70,7,4,6,...,7,6,7,correct,6,6,56,15,6,1142
4,-1,Sib61,wrong-1,56,0,,,,,,...,6,5,7,correct,7,6,55,57,4,354


In [59]:
df = pd.concat([df_v1, df_v2, df_v3], axis=0, ignore_index=True)
# df["is-journalist"] = False
df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
df["workDurationInSeconds"] = df["workDurationInSeconds.main"] + df["workDurationInSeconds.qualification"]

/tmp/ipykernel_73291/2953061995.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["POINTS"] = df["POINTS.main"].astype(int) + df["POINTS.qualification"].astype(int)
/tmp/ipykernel_73291/2953061995.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["workDurationInSeconds"] = df["workDurationInSeconds.main"] + df["workDurationInSeconds.qualification"]


In [60]:
columns_to_rename = {
  "swymerId": "METADATA.swymerId",
  "workDurationInSeconds.main": "METADATA.workDurationInSeconds.main",
  "workDurationInSeconds.qualification": "METADATA.workDurationInSeconds.qualification",
  "workDurationInSeconds": "METADATA.workDurationInSeconds",
  "personal-code": "METADATA.personal-code",
  # "is-journalist": "JOURNALIST",
  "METADATA.FEATURE": "FEATURE",
}

df = df.rename(columns=columns_to_rename)

In [61]:
# order columns alphabetically
df = df.reindex(sorted(df.columns), axis=1)

In [62]:
from pprint import pprint

columns = list(df.columns)
pprint(columns)

['FEATURE',
 'METADATA.GROUP',
 'METADATA.personal-code',
 'METADATA.swymerId',
 'METADATA.workDurationInSeconds',
 'METADATA.workDurationInSeconds.main',
 'METADATA.workDurationInSeconds.qualification',
 'POINTS',
 'POINTS.main',
 'POINTS.qualification',
 'ai-system-evaluation.additional-functionality',
 'ai-system-evaluation.benefit-of-doubt',
 'ai-system-evaluation.classification-comprehensible',
 'ai-system-evaluation.easy-to-follow-what-system-does',
 'ai-system-evaluation.know-what-will-happen-next-time',
 'ai-system-evaluation.most-useful-explanation-feature',
 'ai-system-evaluation.other-information-wish',
 'ai-system-evaluation.rely-on-ai-system',
 'ai-system-evaluation.sceptical-about-ai-system',
 'ai-system-evaluation.system-able-to-classify-news-articles',
 'ai-system-evaluation.system-able-to-detect-SDG',
 'ai-system-evaluation.system-can-correctly-classify-news-articles',
 'ai-system-evaluation.system-can-efficiently-classify-news-articles',
 'ai-system-evaluation.system-

In [63]:
# determine which columns only contain values which can be parsed to numbers
def is_number(s):
    if s == "":
        return True # empty cells are treated as possible numbers
    try:
        float(s)
        return True
    except ValueError:
        return False

In [64]:
# determine which columns only contain values which can be parsed to integers or are empty
# change data type of numeric columns to int
numeric_columns = df.columns[df.applymap(is_number).all()]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, downcast='integer')

/tmp/ipykernel_73291/1070754137.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  numeric_columns = df.columns[df.applymap(is_number).all()]


In [65]:
rating_columns = [col for col in df.columns if ".rating-" in col]

In [26]:
df

,FEATURE,METADATA.GROUP,METADATA.personal-code,METADATA.swymerId,METADATA.workDurationInSeconds,METADATA.workDurationInSeconds.main,METADATA.workDurationInSeconds.qualification,POINTS,POINTS.main,POINTS.qualification,...,newsitem.7.system-evaluation.xai-features-useful,newsitem.8.control-question,newsitem.8.rating-after-xai,newsitem.8.rating-before-xai,newsitem.8.system-evaluation.classified-correctly,newsitem.8.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.system-evaluation.indications-useful,newsitem.8.system-evaluation.indications-useful-concreteness,newsitem.8.system-evaluation.understand-what-system-does,newsitem.8.system-evaluation.xai-features-useful
0,basic,2,19713,CX_79169039921,258,178,80,3,3,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,basic,2,svm73,CX_31975747433,1721,432,1289,2,2,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,basic,2,VJT09,CX_381218321691,1752,1178,574,5,5,0,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,basic,2,SKI877,CX_42129048083,1920,1127,793,7,5,2,...,6.0,correct,-1.0,-1.0,6.0,6.0,6.0,6.0,6.0,6.0
4,basic,2,AHL41,CX_13462817447,2483,702,1781,8,6,2,...,7.0,correct,-1.0,-1.0,7.0,7.0,7.0,7.0,7.0,7.0
5,salient,2,SJS97,CX_40555030233,1434,796,638,8,6,2,...,6.0,correct,-1.0,-1.0,7.0,4.0,6.0,4.0,6.0,4.0
6,salient,2,FDA73,CX_97763898065,5217,3610,1607,7,5,2,...,7.0,correct,-1.0,-1.0,3.0,6.0,3.0,5.0,6.0,2.0
7,salient,2,jrc88,CX_09727584668,1228,691,537,8,6,2,...,7.0,correct,-1.0,-1.0,7.0,7.0,7.0,7.0,7.0,7.0
8,salient,2,JDP35,CX_65866460288,1257,590,667,6,4,2,...,7.0,correct,-1.0,55.0,7.0,6.0,7.0,6.0,7.0,7.0
9,salient,2,DLL27,CX_61956114485,1653,980,673,8,6,2,...,6.0,correct,-1.0,-1.0,2.0,6.0,6.0,2.0,6.0,2.0


In [72]:
# check all value in rating columns which are < 0 with 0
print((df[rating_columns] < -1).eq(False).all())


newsitem.1.rating-after-xai     True
newsitem.1.rating-before-xai    True
newsitem.2.rating-after-xai     True
newsitem.2.rating-before-xai    True
newsitem.3.rating-after-xai     True
newsitem.3.rating-before-xai    True
newsitem.4.rating-after-xai     True
newsitem.4.rating-before-xai    True
newsitem.5.rating-after-xai     True
newsitem.5.rating-before-xai    True
newsitem.6.rating-after-xai     True
newsitem.6.rating-before-xai    True
newsitem.7.rating-after-xai     True
newsitem.7.rating-before-xai    True
newsitem.8.rating-after-xai     True
newsitem.8.rating-before-xai    True
dtype: bool


In [73]:

# check all value in rating columns which are > 100 with 100
print((df[rating_columns] > 100).eq(False).all())

newsitem.1.rating-after-xai     True
newsitem.1.rating-before-xai    True
newsitem.2.rating-after-xai     True
newsitem.2.rating-before-xai    True
newsitem.3.rating-after-xai     True
newsitem.3.rating-before-xai    True
newsitem.4.rating-after-xai     True
newsitem.4.rating-before-xai    True
newsitem.5.rating-after-xai     True
newsitem.5.rating-before-xai    True
newsitem.6.rating-after-xai     True
newsitem.6.rating-before-xai    True
newsitem.7.rating-after-xai     True
newsitem.7.rating-before-xai    True
newsitem.8.rating-after-xai     True
newsitem.8.rating-before-xai    True
dtype: bool


In [74]:
df.to_csv("data/data_cleaned.csv", index=False)